<h1>Host and Query Model with Grid</h1>
<p>In this series of tutorials we show how you can serve and query models on Grid platform.</p>

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:5000)
 - Grid Node Alice (http://localhost:3000)
 - Grid Node Bob (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

<h3>Import dependencies</h3>

In [2]:
import syft as sy
import grid as gr
import torch as th

<h3>Set up configs</h3>
<p>Now, we'll set our syft hook and instantiate our grid</p>

In [3]:
hook = sy.TorchHook(th)

In [4]:
my_grid = gr.GridNetwork("http://localhost:5000")

<h3>Create a test model</h3>
<p>For test purposes, we'll create a toy model with random values as parameters</p>

In [5]:
toy_model = th.nn.Linear(2, 5)
data = th.zeros((5,2))
traced_model = th.jit.trace(toy_model, data)

<h3> Hosting our models</h3>
<p>Now,we'll host our models on grid nodes registered on our grid network. For test purposes, we'll register the same model (can be jit model or plan model) changing only the model_id. </p>  
<p>Nowadays, the choose of grid node that will host the model is random, but in the future we can add a better metric to do this.</p>

In [6]:
my_grid.host_model(traced_model, "MNIST")

In [8]:
my_grid.host_model(traced_model, "CIFAR10")

In [11]:
my_grid.host_model(traced_model, "SKINCANCER")

<h3>Querying/Running inference</h3>
<p>Now, we want to recover the reference of our models registered previously.</p>
<p>It is important to notice that we can have the same ID on different nodes (we can have different versions of same model distributed throughout our network). To solve this problem, when we perform a query, it will return a list of grid nodes that contains some version of this specific model.</p>

In [12]:
mnist_workers = my_grid.query_model("MNIST")
first_mnist_host = mnist_workers[0]
print("HOST ID: ", first_mnist_host.id)
print("HOST ADDRESS: ", first_mnist_host.uri)
toy_data = th.zeros((5,2))
first_mnist_host.run_inference(model_id="MNIST", data=toy_data)

HOST ID:  Bob
HOST ADDRESS:  http://localhost:3001


{'prediction': [[0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608]]}

In [16]:
mnist_workers = my_grid.query_model("CIFAR10")
first_mnist_host = mnist_workers[0]
print("HOST ID: ", first_mnist_host.id)
print("HOST ADDRESS: ", first_mnist_host.uri)
toy_data = th.zeros((5,2))
first_mnist_host.run_inference(model_id="CIFAR10", data=toy_data)

HOST ID:  Bob
HOST ADDRESS:  http://localhost:3001


{'prediction': [[0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608]]}

In [14]:
mnist_workers = my_grid.query_model("SKINCANCER")
first_mnist_host = mnist_workers[0]
print("HOST ID: ", first_mnist_host.id)
print("HOST ADDRESS: ", first_mnist_host.uri)
toy_data = th.zeros((5,2))
first_mnist_host.run_inference(model_id="SKINCANCER", data=toy_data)

HOST ID:  Alice
HOST ADDRESS:  http://localhost:3000


{'prediction': [[0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608],
  [0.16302770376205444,
   -0.23383474349975586,
   0.33468955755233765,
   -0.24720534682273865,
   0.3982263207435608]]}

<h3>Disconnect Nodes</h3>

In [15]:
my_grid.disconnect_nodes()